In [1]:
import pandas as pd
import numpy as np
from pyecharts.charts import Map,Bar
from pyecharts import options as opts

In [2]:
#读取数据
login_data = pd.read_csv(r'C:\Users\Administrator\Desktop\TIPMA\login.csv'
                         ,encoding='gbk')
study_information_data = pd.read_csv(r'C:\Users\Administrator\Desktop\TIPMA\study_information.csv'
                                     ,encoding='gbk')
users_data = pd.read_csv(r'C:\Users\Administrator\Desktop\TIPMA\users.csv'
                     ,encoding='gbk',thousands=',')    #thousands参数是去掉千位符

### task1.1进行缺失值、重复值等方面的处理

1.分别导入三个csv文件，检查是否存在缺失值，重复行

2.存在缺失值，则使用填充，存在重复行，则删除重复行

#### Step1.处理login数据集(无重复数据，无缺失值)

In [3]:
#判断login数据集中是否有重复数据
login_data[login_data.duplicated()]

,user_id,login_time,login_place


In [4]:
#判断login数据集是否有缺失值
login_data.isnull().any()

user_id        False
login_time     False
login_place    False
dtype: bool

#### Step2.处理study_information数据集(无重复数据，有缺失值)

In [5]:
#判断study_information数据集中是否有重复数据
study_information_data[study_information_data.duplicated()]

,user_id,course_id,course_join_time,learn_process,price


In [6]:
#判断study_information数据集是否有缺失值
study_information_data.isnull().any()

user_id             False
course_id           False
course_join_time    False
learn_process       False
price                True
dtype: bool

In [7]:
#study_information数据集存在缺失值，使用0填充
study_information_data.fillna(value = 0,inplace = True)

In [8]:
#处理完的数据导出csv文件
pd.DataFrame.to_csv(study_information_data
                    ,r'C:\Users\Administrator\Desktop\taskA\task1_1_2.csv'
                    ,','
                    ,index=False)

#### Step3.处理users数据集(有重复数据，有缺失值)

In [9]:
#判断users数据集中是否有重复数据
users_data[users_data.duplicated()]

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school
3429,用户40732,2020/5/25 18:01,--,1,0,0.00,NaN
3535,用户36488,2020/5/22 15:16,2020/6/17 8:25,1,0,1006.88,NaN
36098,用户7909,2019/4/9 15:08,2019/5/1 8:14,0,0,96.85,NaN


In [10]:
#users数据集存在重复数据，去重处理
users_data.drop_duplicates(inplace = True , ignore_index = True)

In [11]:
#判断users数据集是否有缺失值,（user_id和school存在缺失值）
users_data.isnull().any()

user_id                    True
register_time             False
recently_logged           False
number_of_classes_join    False
number_of_classes_out     False
learn_time                False
school                     True
dtype: bool

In [12]:
#把user_id为缺失的数据过滤掉
users_data = users_data[users_data.user_id.notnull()]

In [13]:
#处理完的数据导出csv文件
pd.DataFrame.to_csv(users_data
                    ,r'C:\Users\Administrator\Desktop\taskA\task1_1_3.csv'
                    ,','
                    ,index=False)

### task1.2对于处理 recently_logged 字段的“--”值

1.在users数据集中筛选出recently_logged 字段的“--”的数据

2.判断这部分数据中的user是否是第一次注册

3.如果是第一次注册，则recently_logged = registration_time + learn_time，否则删除数据

In [14]:
# 1.在users数据集中筛选出recently_logged 字段的“--”的数据
users_null_data = users_data[users_data.recently_logged == '--']

In [15]:
# 2.判断这部分用户是否是第一次注册(可以发现 recently_logged =“--”的这部分用户都是第一次注册的)
users_nonull_list = users_data[users_data.recently_logged != '--'].user_id.tolist()
users_null_data[users_null_data['user_id'].isin(users_nonull_list)]

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school


In [16]:
# 3.recently_logged 字段的“--”,使用recently_logged = registration_time + learn_time
f1 = lambda x :pd.Timedelta(x,unit='m')
users_data['recently_logged_pic'] = pd.to_datetime(users_data['register_time']) \
+ users_data['learn_time'].astype('float64').apply(f1)

f2 = lambda x:x.strftime('%Y/%m/%d %H:%M')
users_data['recently_logged_pic'] = users_data['recently_logged_pic'].apply(f2)

for i in range(len(users_data)):
    if users_data.iloc[i,2] == '--':
        users_data.iloc[i,2] = users_data.iloc[i,7]
        
users_data.drop(columns='recently_logged_pic',inplace=True)
users_data

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school
0,用户44251,2020/6/18 9:49,2020/6/18 9:49,0,0,41.25,NaN
1,用户44250,2020/6/18 9:47,2020/6/18 9:48,0,0,0.00,NaN
2,用户44249,2020/6/18 9:43,2020/6/18 9:43,0,0,16.22,NaN
3,用户44248,2020/6/18 9:09,2020/6/18 9:09,0,0,0.00,NaN
4,用户44247,2020/6/18 7:41,2020/6/18 8:15,0,0,1.80,NaN
...,...,...,...,...,...,...,...
43975,用户6,2018/9/11 16:13,2018/9/11 16:14,0,0,0.00,NaN
43976,用户5,2018/9/10 15:48,2020/6/15 17:13,9,5,2116.15,NaN
43977,用户4,2018/9/10 14:15,2020/6/5 9:50,4,0,341.20,NaN
43978,用户3,2018/9/4 13:32,2020/6/18 9:18,2,1,370.35,NaN


### task2.1绘制各省份各城市登陆次数的热力地图，分析用户分布情况

1.由于login数据集中login_place字段杂乱无章，所以自定义省份名称进行匹配，以方便画地图

2.得到各省份的登陆次数的汇总数据，使用pyecharts绘制热力地图

In [17]:
#自定义省份名称
provinces = ['河北','山西','辽宁','吉林','黑龙江','江苏','浙江','安徽','福建'
            ,'江西','山东','河南','湖北','湖南','广东','海南','四川','贵州'
            ,'云南','陕西','甘肃','青海','台湾','内蒙古','广西','西藏','宁夏'
            ,'新疆','北京','天津','上海','重庆','香港','澳门']

#添加一个province用来存放对应的省份
login_data['province'] = np.nan

#找到对应的省份
for province in provinces:
    for i in range(len(login_data)):
        if province in login_data.iloc[i,2]:
            login_data.iloc[i,3] = province

#取出有省份信息的数据
login_province = login_data[login_data.province.notnull()]

#汇总各省份的登陆次数
province_number = login_province.groupby(by='province').agg({'province':'count'})

#绘制热力地图
max_login = province_number.province.max()
map1 = Map()
map1.add(""
         ,[list(z) for z in zip(province_number.index.tolist(),province_number.province.tolist())]
         ,maptype='china')
map1.set_global_opts(title_opts=opts.TitleOpts(title='各省份登陆次数热力地图')
                     ,visualmap_opts=opts.VisualMapOpts(max_=max_login))

map1.render_notebook()
#map1.render()

C:\Users\Administrator\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


### task2.2分别绘制工作日与非工作日各时段的用户登录次数柱状图

1.区分login数据工作日和非工作日，各时段

2.分别绘制工作日和非工作日柱形图

In [18]:
#将login_time分开日期，时间
login_data['login_min'] = login_data['login_time'].apply(lambda x:x.split(' ')[1])
login_data['login_day'] = login_data['login_time'].apply(lambda x:x.split(' ')[0])

#新建一列用来存放时间段
login_data['time_bins'] = np.nan

#自定义时间段
time_label = [['凌晨','01:00:00','04:59:59']
              ,['早上','05:00:00','07:59:59']
              ,['上午','08:00:00','10:59:59']
              ,['中午','11:00:00','12:59:59']
              ,['下午','13:00:00','16:59:59']
              ,['傍晚','17:00:00','18:59:59']
              ,['晚上','19:00:00','22:59:59']
              ,['子夜','23:00:00','23:59:59']
              ,['子夜','00:00:00','00:59:59']]

#根据login_min时间匹配对应的时间段
for num in range(len(login_data)):
    for label in time_label:
        if login_data.iloc[num,4] >= label[1] and login_data.iloc[num,4] <= label[2]:
            login_data.iloc[num,6] = label[0]
            
#判别工作日,非工作日
login_data['how_data'] = pd.to_datetime(login_data['login_day']).dt.dayofweek + 1
login_data['is_workday'] = login_data['how_data'].apply(lambda x:'1' if x<=5 else '0')

In [19]:
#分别筛选出工作日和非工作日的数据
login_is_workday = login_data[login_data['is_workday'] == '1']
login_no_workday = login_data[login_data['is_workday'] == '0']

#绘制工作日各时段的用户登录次数柱形图
workday_groupby = login_is_workday.groupby(by='time_bins').agg({'time_bins':'count'})
max_workday = workday_groupby.time_bins.max()

bar1 = Bar(init_opts=opts.InitOpts(width='680px',height='450px'))
bar1.add_xaxis(workday_groupby.index.tolist())
bar1.add_yaxis("",workday_groupby.time_bins.tolist())
bar1.set_global_opts(title_opts=opts.TitleOpts(title='工作日各时段的用户登录次数')
                     ,visualmap_opts=opts.VisualMapOpts(max_=max_workday))

bar1.render_notebook()

C:\Users\Administrator\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [20]:
#绘制非工作日各时段的用户登录次数柱形图
noworkday_groupby = login_no_workday.groupby(by='time_bins').agg({'time_bins':'count'})
max_noworkday = noworkday_groupby.time_bins.max()

bar2 = Bar(init_opts=opts.InitOpts(width='680px',height='450px'))
bar2.add_xaxis(noworkday_groupby.index.tolist())
bar2.add_yaxis("",noworkday_groupby.time_bins.tolist())
bar2.set_global_opts(title_opts=opts.TitleOpts(title='非工作日各时段的用户登录次数')
                     ,visualmap_opts=opts.VisualMapOpts(max_=max_noworkday))

bar2.render_notebook()

C:\Users\Administrator\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


### task2.3计算平台用户的流失率

In [21]:
#users数据中取用户最近的访问日期
users_logged_max = users_data.groupby(by='user_id').agg({'recently_logged':'max'})
users_logged_max['begin_data'] = '2020/6/18'

#计算最近的访问日期和采集截至时间2020/6/18的间隔天数
users_logged_max['logged_day'] = (pd.to_datetime(users_logged_max['begin_data'])\
                                  - pd.to_datetime(users_logged_max['recently_logged'])).dt.days

#计算流失人数wastage，流失率wastage_rate
wastage = len(users_logged_max[users_logged_max.logged_day > 90])
allnum = len(users_logged_max)
wastage_rate = wastage/allnum
wastage_rate

0.5818757401840211

### task2.4 自由发挥

### task3.1计算受欢迎程度前十的课程，绘制柱形图

In [22]:
#判断user_id和course_id两列是否有重复数据
study_information_data[study_information_data[['user_id','course_id']].duplicated()]

,user_id,course_id,course_join_time,learn_process,price


In [23]:
#汇总每一门课的人数
course_num = study_information_data[['user_id','course_id']].\
groupby(by='course_id').agg({'user_id':'count'})

#计算每一门课的受欢迎程度y
course_num['y'] = round((course_num - course_num.min())/(course_num.max() - course_num.min()),2)

#取出前十受欢迎程度的课程
course_top10 = course_num.sort_values(by='y',ascending=False).y.head(10)
course_top10

course_id
课程76     1.00
课程31     0.72
课程17     0.64
课程191    0.54
课程180    0.47
课程52     0.46
课程34     0.43
课程171    0.41
课程50     0.40
课程26     0.36
Name: y, dtype: float64

In [24]:
#绘制前十受欢迎程度的课程柱形图
bar2 = Bar()
bar2.add_xaxis(course_top10.index.tolist())
bar2.add_yaxis("",course_top10.values.tolist())
bar2.set_global_opts(title_opts=opts.TitleOpts(title='最受欢迎的前 10 门课程')
                     ,visualmap_opts=opts.VisualMapOpts(max_=1))

bar2.render_notebook()

C:\Users\Administrator\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


### task3.2使用基于物品的协同过滤算法为用户进行课程推荐

In [25]:
#把study_information_data的用户，课程列拿出来，生成一个用户-课程矩阵（隐式反馈）
user_course_df = study_information_data[['user_id','course_id']]
user_course_df['score'] = 1
user_course = user_course_df.pivot(index='user_id',columns='course_id',values='score')
user_course.fillna(value=0,inplace=True)

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
#计算课程相似度
def courses_similar(user_course):
    #转化为ndarray，方便运算
    user_course_nd = np.array(user_course.T)
    m = user_course_nd.shape[0]
    
    #course_sim存放课程相似度矩阵
    course_sim = np.zeros((m,m))
    for i in range(m):
        for j in range(m):
            s1 = np.dot(user_course_nd[i],user_course_nd[j])
            s2 = np.sqrt(sum(user_course_nd[i])*sum(user_course_nd[j]))
            course_sim[i][j] = round(s1/s2,4)
    return course_sim

#top5的用户的user_id
def user_top5(study_information_data):
    #取出总学习进度top5的用户的user_id
    study_information_data['learn_process_rate'] = study_information_data['learn_process'].str.extract(r'(\d+)')
    study_information_data['learn_process_rate'] = study_information_data['learn_process_rate'].astype('float64')
    learn_process_rank = study_information_data.groupby(by='user_id').agg({'learn_process_rate':'sum'})
    learn_process_rank5 = learn_process_rank.sort_values(by=['learn_process_rate'],ascending=False).head(5)
    user_id = learn_process_rank5.index.tolist()
    
    return user_id

#产生推荐
def recommend_course(user_id,course_sim):
    #取出该user_id学习的course_id
    user_courseid = study_information_data[study_information_data['user_id'] == user_id]['course_id'].tolist()

    #在课程相似度矩阵中筛选出course_id与对应的user_id没学习过的课程course_id的相似度矩阵
    course_sim_df = pd.DataFrame(course_sim,index=user_course.columns,columns=user_course.columns)
    course_sim_df1 = course_sim_df.reset_index()
    course_sim_df2 = course_sim_df1[course_sim_df1['course_id'].isin(user_courseid)]
    course_sim_df2.set_index('course_id',inplace=True)
    course_sim_df2.drop(columns=user_courseid,inplace=True)  #删除user_id学习过的course_id
    
    #产生user_id的推荐列表
    course_sim_df2.apply(lambda x:x.sum())
    all_recommend = course_sim_df2.apply(lambda x:x.sum())
    recommend = all_recommend.sort_values(ascending=False).head(3)
    all_recommend = course_sim_df2.apply(lambda x:x.sum())
    recommend_Series = all_recommend.sort_values(ascending=False).head(3)
    recommend = pd.DataFrame(recommend_Series).reset_index()
    recommend['user_id'] = user_id
    recommend.columns = pd.Series(['course_id','pic_score','user_id'])
    
    return recommend

if __name__ == '__main__':
    course_sim = courses_similar(user_course)
    user_id = user_top5(study_information_data)
    usertop5_recommend = pd.DataFrame()
    for user in user_id:
        usertop5_recommend = usertop5_recommend.append(recommend_course(user,course_sim))
    print(usertop5_recommend)

C:\Users\Administrator\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


  course_id  pic_score  user_id
0     课程148    15.4079   用户1193
1     课程162    15.3230   用户1193
2      课程40    14.5287   用户1193
0     课程141    12.0527  用户13841
1     课程158    11.8050  用户13841
2     课程148    11.5723  用户13841
0      课程40    15.7472  用户32684
1      课程96    12.2334  用户32684
2     课程180    12.1616  用户32684
0      课程40    15.3016  用户36989
1     课程180    11.8898  用户36989
2      课程52    11.6056  用户36989
0     课程180    13.0323  用户24985
1      课程52    12.6867  用户24985
2      课程50    11.8809  用户24985


### task3.3 自由发挥